In this note book we provide some examples of how to define local, nonlocal, single and multi-channel interactions.

In [15]:
import numpy as np

from jitr import reactions, rmatrix
from jitr.reactions.potentials import coulomb_charged_sphere, woods_saxon_potential
from jitr.utils import delta, kinematics

In [16]:
# a local scalar interaction
def interaction_local(r, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(r, V0, W0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return nuclear + coulomb

In [19]:
# a nonlocal scalar interaction
def interaction_nonlocal(r, rp, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(0.5 * (r + rp), V0, W0, R0, a0) * np.exp(
        (r - rp) ** 2
    )
    coulomb = coulomb_charged_sphere(r, R0, Zz)
    return nuclear + coulomb

In [20]:
def interaction_2level(r, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(r, V0, W0, R0, a0)
    nuclear1 = woods_saxon_potential(r, V1, W1, R0, a0)
    coupling = woods_saxon_potential(r, g, 0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return np.array([[nuclear0 + coulomb, coupling], [coupling, nuclear1 + coulomb]])

In [21]:
def interaction_2level_nonlocal(r, rp, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(r, V0, W0, R0, a0) * np.exp(-((r - rp) ** 2))
    nuclear1 = woods_saxon_potential(r, V1, W1, R0, a0) * np.exp(-((r - rp) ** 2))
    coupling = woods_saxon_potential(r, g, 0, R0, a0) * np.exp(-((r - rp) ** 2))
    return np.array([[nuclear0 + coulomb, coupling], [coupling, nuclear1 + coulomb]])

define system

In [22]:
Elab = 35  # MeV
Ca48 = (28, 20)
proton = (1, 1)

In [23]:
sys = reactions.ProjectileTargetSystem(
    channel_radii=np.array([5 * np.pi]),
    l=np.array([0]),
    mass_target=kinematics.mass(*Ca48),
    mass_projectile=kinematics.mass(*proton),
    Ztarget=Ca48[1],
    Zproj=proton[1],
)
Ecm, mu, k, eta = kinematics.classical_kinematics(
    sys.mass_target,
    sys.mass_projectile,
    Elab,
    sys.Ztarget * sys.Zproj,
)
channels, asymptotics = sys.coupled(Ecm, mu, k, eta)

In [24]:
solver = rmatrix.Solver(nbasis=40)

In [25]:
params = (42.0, 18.1, 4.8, 0.7, sys.Zproj * sys.Ztarget)
R, S, uext_boundary = solver.solve(
    channels, asymptotics, interaction_local, local_args=params
)

In [26]:
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

phase shift: -70.751 + i (-3.439) [degrees]
